<a href="https://colab.research.google.com/github/mrsferret/Code-Division/blob/main/Worksheets/15_2_Decision_Tree_coded_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding a simple decision tree
---

In this worksheet we are going to work with a data set, using the idea of a decision tree class.  We are going to simplify the model and use Python code to make a simple decision tree classification model.  We will do this for two reasons:
*   writing the code is often good for helping to understand what is going on under the bonnet of a library function
*   it is a good coding exercise for practice as it mostly depends on calculations and if..elif..else statements

In this worksheet we are going to code a decision tree which will use the calculated probabilities to make decisions about whether a row of given data would be classified as Iris-virginica, or not, based on sepal and petal dimensions.  It is easier to classify between two values (Iris-virginica or not).  Later, using this information, species would be further predicted by probabilities of error.

![Iris-petals and sepals](https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg)

The workflow is:
*  divide the data set into 70% of the rows for training and 30% for testing  (we can increase the size of the training set later)
*  find the mean for each of the 4 size columns
*  calculate the proportion of each column that are on or above mean that are of a species (ie proportion of petal-lengths on or above mean that are Iris-virginica)
*  infer the proportion of each that are not of that species (using 1 - proportion above).  In both cases we are looking to find if either of these is 1, which could be infered as definitely not that species. 
*  calculate a Gini Index that will indicate the probability that a prediction will be incorrect
*  use the results of the Gini Index to model a decision tree
*  code the decision tree model into a function that will return whether or not a row in the test set is predicted to be of species Iris-virginica
*  use the decision tree function to predict, for each row in the test set, if the species will be Iris-virginicia or not, using a set of nested if statements to classify
*  compare the predicted values against the actual values in the test set - what proportion were predicted correctly?


### Exercise 1 - investigate the iris data set
---
Let's start by looking at the data.  We are going to use a data set that contains data on iris flowers.

Read the data at this location: https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv into a dataframe called iris_data

The columns in the CSV file do not have headings, when you read the file, add column headings like this:
```
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, name=names)
```
*  Take a look at the column info (how many columns, what type of data, any missing data?)
*  Take a look at the data values in the first 10 and the last 10 records to get an idea of the type of values included
*  Find out how many unique values there are in the species column
*  Find out the maximum, minimum, mean, median and upper and lower quartile values in each of the columns


In [12]:
import pandas as pd
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, names=names)

print(iris_data.describe())
print(iris_data.info())
print(iris_data.head(10))
print(iris_data.tail(10))
count_unique = iris_data['species'].nunique()                    
print("Count of unique values in species: ", count_unique) 



       sepal-length  sepal-width  petal-length  petal-width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.054000      3.758667     1.198667
std        0.828066     0.433594      1.764420     0.763161
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal-length  150 non-null    float64
 1   sepal-width   150 non-null    float64
 2   petal-length  150 non-null    float64
 3   petal-width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
Non

### Exercise 2 - split the data into train and test sets
---

Split the data set into and 70% train, 30% test, split.  From now, just use the train data set.


In [29]:
# import the train_test_split function
from sklearn.model_selection import train_test_split

# create the classification variables from the all columns
train, test = train_test_split(iris_data, test_size=0.30)
print("\ntrain: ", train.describe())
print("\ntest: ", test.describe())


train:         sepal-length  sepal-width  petal-length  petal-width
count    105.000000   105.000000    105.000000    105.00000
mean       5.793333     3.086667      3.601905      1.14381
std        0.820795     0.434136      1.779422      0.77023
min        4.300000     2.000000      1.000000      0.10000
25%        5.100000     2.800000      1.500000      0.30000
50%        5.800000     3.000000      4.200000      1.30000
75%        6.400000     3.400000      5.100000      1.80000
max        7.900000     4.400000      6.900000      2.50000

test:         sepal-length  sepal-width  petal-length  petal-width
count     45.000000    45.000000     45.000000    45.000000
mean       5.960000     2.977778      4.124444     1.326667
std        0.842453     0.427407      1.691980     0.738980
min        4.400000     2.200000      1.300000     0.100000
25%        5.400000     2.700000      3.300000     1.000000
50%        5.800000     3.000000      4.500000     1.300000
75%        6.600000    

### Exercise 3 - assumptions and classification
---

Let's make some assumptions based on the data

1.  Iris-setosa, Iris-versicolor, Iris-virginica are the full range of types of iris to be analysed
2.  Although this is a small data set, the means are fairly representative

With these in mind, let's start by classifying sepal/petal size into long/short and wide/narrow with values on or above the mean taken as long or wide and those below as short or narrow.

This is a starting point.  We will be trying to find a value (indicator) for each column where rows on or above do not contain any of a particular species, this might indicate that this column is a good (if not rough) indicator of species.  For now, the indicator is the mean.

*  Drop any null values from each column

Calculate, and store the means of the four columns

*  **Test**:
Display train.describe() to see the value of the means of the training set. Print the four means and compare to the output of train.describe() to check that they have been calculated correctly.

*  Create a new dataframe with the numeric columns encoded so show a 1 for any value that is above the mean for its column and 0 for any that isn't.




In [30]:
# get the mean for each column and apply a function to encode into 1 (above mean) and 0 (mean or below mean)
#=======================================================================
# encode each column, where values greater than or 
# equal to median = 1 and values less than median = 0
#=======================================================================
def encode(df, **kwds):
  #display("df: ", df)
  mean = kwds['indicator']
  #display("mean: ", mean)
  key = kwds['key']

  if df[key] >= mean:
    return 1
  else:
    return 0

#===============================================================================
# Start here
#===============================================================================
sepal_length_mean = iris_data['sepal-length'].mean()
sepal_width_mean = iris_data['sepal-width'].mean()
petal_length_mean = iris_data['petal-length'].mean()
petal_width_mean = iris_data['petal-width'].mean()

# run the function for each column so that each of the four columns are encoded, then drop the original columns, saving as a new dataframe
train['sepal-width'] = train.apply(encode, axis=1, key='sepal-width', indicator=train['sepal-width'].mean())
train['sepal-length'] = train.apply(encode, axis=1, key='sepal-length', indicator=train['sepal-length'].mean())
train['petal-width'] = train.apply(encode, axis=1, key='petal-width', indicator=train['petal-width'].mean())
train['petal-length'] = train.apply(encode, axis=1, key='petal-length', indicator=train['petal-length'].mean())
print("\ntrain: \n", train.describe())
print("\ntrain: \n", train.head())

#median = get_median(covid_vax_df)
#covid_vax_df["encoded"] = covid_vax_df.apply(encode_daily, axis = 1, key="daily_vaccinations_per_million", median=median)




train: 
        sepal-length  sepal-width  petal-length  petal-width
count    105.000000   105.000000    105.000000   105.000000
mean       0.514286     0.485714      0.600000     0.561905
std        0.502193     0.502193      0.492248     0.498533
min        0.000000     0.000000      0.000000     0.000000
25%        0.000000     0.000000      0.000000     0.000000
50%        1.000000     0.000000      1.000000     1.000000
75%        1.000000     1.000000      1.000000     1.000000
max        1.000000     1.000000      1.000000     1.000000

train: 
      sepal-length  sepal-width  petal-length  petal-width          species
79              0            0             0            0  Iris-versicolor
28              0            1             0            0      Iris-setosa
110             1            1             1            1   Iris-virginica
114             1            0             1            1   Iris-virginica
39              0            1             0            0      Ir

### Exercise 4 - Calculate the proportion of values on or above the mean that are of each species

We are going to focus on the `Iris-virginica` species first.

First we will calculate, for each dimension column (`sepal-length, sepal-width, petal-length, petal-width`) what proportion of values in that column, where the value is on or above the mean, are classified as `Iris-virginica`.

We will do this by filtering all the records in each column of the the `train` set that are on or above the mean and match the species .  Then use the outcome to calculate the proportion of the full `train` set for which a value on or above the mean that are of species `Iris-virginica`.

*  filter for values in the `sepal-length` column being on or above the mean and the species column being `Iris-virginica`.  Then divide the count of rows in this filtered dataset by the count of rows in a second data set, filtered for just the value being on or above the mean.

*  Do this for all four columns, for `Iris-virginica`  (4 operations).

Print the results to see which columns look like they might most reliably predict the species as `Iris-virginica` (the result is as close as possible to 1).  The highest numbers may indicate the most reliable indicators, but we will do some more before coming to this conclusion.

*  By definition, those on or above the mean that are NOT Iris_virginica will be `1 - the proportion of those that are.  Calculate these

The first one has been done for you.

*  We will also need the proportion of those BELOW the mean that are NOT Iris-virginica.  Calculate these in the same way



In [38]:
# calculate the proportion of results where the value is on or above mean that are of the species Iris-virginica
def myFunc(x):

  if x == 0:
    return False
  else:
    return True

#==============================================================================
# start
#==============================================================================
# process col 'sepal-width'
#==============================================================================
filtered = filter(myFunc, train['sepal-width'])
print("filtered: ", filtered)
#count = filtered.count()
# counting rows
#print(len(filtered.index))
#print("\nfiltered count = ", count)
#for x in filtered:
#  print(x)

#def myFunc(x):
#  if x < 1:
#    return False
#  else:
#    return True

#adults = filter(myFunc, ages)

#for x in adults:
#  print(x)


filtered:  <filter object at 0x7f5c8abb3e10>


### Exercise 5 - Calculate the proportion of each column, where the value is below mean, that are of species `Iris-virginica`

Repeat the code above, this time looking for values below the mean

In [ ]:
# calculate the proportion of results where the value is below mean that are of the species Iris-virginica



### Exercise 5 - calculate for the other two Iris species
---

Do the same calculations for the Iris-versicolor species, then for the Iris-setosa species.






### Exercise 6 - predict from the results
---

Create a list of dictionaries from the results Exercise 4 and 5 (e.g. {'species':..., 'above_mean': 0.xx, 'below_mean': 0.xx}  

Then use a loop to go through the list and print:  
*  any species and indicator (above or below mean) that can reliably be predicted.  A reliable prediction may be one over 0.5

In [ ]:
# show which columns are reliable predictors



### Exercise 6 - Make a decision tree
---

Use pencil and paper or a graphical application to create a decision tree for Iris-virginica, using the following rules (use the picture below as a guide):

*  The column with the highest indicator is placed at the top
*  Other columns are placed in order below
*  The rest of the columns are placed in order below these

Any column where one branch (on or above mean OR below mean) has an indicator of 0, could be classified as a strong indicator of Iris_virginica being the species.  Anything else, unless there is something very close to 0, could be classified as a weak indicator of Iris_virginica being the species.

Let's code the decision tree using the following logic for this decision tree (yours might be slightly different):

![Decision tree](https://drive.google.com/uc?id=1CTo23EHwR2IPCRjcfSyCQsT_oQ5Exwso)

In the decision tree above, there is no certainty below petal-length so our decision tree will only include petal-width and petal-length.




In [ ]:
def get_species(df):
  # ADD CODE HERE TO RETURN None if petal-width is below mean (encoded as 0) or if petal-length is below mean (encoded as 0), otherwise return 'Iris-virginia'


# use the get_species(df) function to predict the species, count how many are predicted correct and use this to calculate the proportion correct
correct = 0
test_size = test.shape[0]
for i in range(0, test_size):
  species = get_species(test.iloc[i])
  if species == test.iloc[i]['species']:
      correct += 1

print ("Proportion correctly identified", correct / test_size) 


### Exercise 7 - change the measure

We are currently using the mean to act as the decision making line.  We can use the decision tree with a different line.

Change the mean values so that you are instead using the median instead for all four columns.  The code should not need changing except for where you calculated the mean.

Run all the code again.  Is the proportion of correct values better this time?   Is the decision tree still appropriate?


What do you notice? (write your answer here)

### Exercise 8 - try different measures
---

Do the same again but with upper quantile, then again with the lower quantile.  Is it making any difference?  Which give the best looking results?

### Exercise 9 - try a different species

Run the mean test again for the Iris-versicolor species.  Again, try some different decision making lines.

What are the results.  Record them in the text cell below:

Write your answers here:  

# New logic introduced in this worksheet:

1.  Adding headings to a CSV if none currently exist
2.  Splitting a data set into train and test sets

In [ ]:
## this type of plot will show the distribution on a chart
from plotnine import *
ggplot(train, aes(x='petal-length', y='petal-width', color='species')) + geom_point() + geom_vline(train, aes(xintercept=train['petal-length'].mean(), color='species')) + geom_hline(train, aes(yintercept=train['petal-width'].mean(), color='species'))